## Adding the Sensors to the Database

This jupyter notebook goes through the steps of loading sensor information to the database. It also generates the scripts for the sensors, so that they can load more data to the database.

1. Load the relevant libraries.

In [30]:
import io

import geomagio
from obspy.core import UTCDateTime

import pandas as pd
import json

2. Load the information about the Frongoch magnetometer to the database. The information is stored in a file called `frongoch_sensor_details.json`.

In [3]:
with open("frongoch_sensor_details.json", 'r') as frg_file:
    frg_info = json.load(frg_file)
    
print(frg_info)

{'sensor_id': 'frg_single_fgm3', 'name': 'Frongoch Farm', 'address': 'B4572, Aberystwyth SY23 3DG, UK', 'UK_grid_ref': 'SN6056882610', 'latitude': 52.423649, 'longitude': -4.0519305, 'elevation': 133, 'coord_sys': 'H', 'raw_dat_params': ['time', 'reading', 'temperature']}


3. Add new sensors.

### Adding the Frongoch magnetometer to the database.

Load the script for loading the sensor.

In [5]:
# Run this cell to see what the code looks like.
%load add_sensor.py

In [22]:
from add_sensor import add_sensor

Add the Frongoch sensor information to the database.

In [5]:
add_sensor(**frg_info)

Successfully added sensor to database!
New sensor script generated. The script can be used to insert raw sensor data to the database using write_raw_dat(). write_mag_dat() can be used to write processed magnetometer data to the database. The script can be found in this directory with the filename: frg_single_fgm3_dat_insert.py
Successfully generated script for inserting sensor readings for frg_single_fgm3!


#### Run tests to see the Frongoch magnetometer script and insert is setup correctly.

This will require the frongoch sensor id, insert script and the database read and writer password (to add and delete tests).

In [ ]:
# Run this cell to see what the code looks like.
%load test_sensor_script.py

In [1]:
from test_sensor_script import run_all_tests
import frg_single_fgm3_dat_insert as frg_insert

Run the tests

In [2]:
# Use the frongoch sensor id and the 
# The username and password for reading and writing to the database is "Readwriter" and "rwpassword".
run_all_tests(frg_info["sensor_id"], frg_insert, "Readwriter", "rwpassword")

Running tests for newly added sensor...

Testing sensor information is in the database.
[]
Checking sensor info in db Failed!

Testing inserting raw sensor data using script.
Successfully written to the database!
Checking raw sensor data in db Failed!

Testing inserting magnetometer data using script.
Successfully written to the database!
Checking magnetometer data in db Failed!

Total number of failed tests: 3


### Adding the Hartland sensor information.

The first step is to retrieve Hartland observatory data. This can be done using the [geomag-algorithms](https://github.com/usgs/geomag-algorithms) database.

In [38]:
input_factory = geomagio.edge.EdgeFactory()
timeseries = input_factory.get_timeseries(
    observatory = 'HAD',
    channels = ('H', 'E', 'Z', 'F'),
    type = 'variation',
    interval = 'minute',
    starttime = UTCDateTime('2016-01-01T00:00:00Z'),
    endtime = UTCDateTime('2016-03-01T00:00:00Z'))

# The output factory expects a string, but we do not want to save to a file.
# Therefore, stream the data into a string buffer.
HAD_buf = io.StringIO("")

output_factory = geomagio.imfjson.IMFJSONWriter()
output_factory.write(
    channels = ('H', 'E', 'Z', 'F'),
    out = HAD_buf,
    timeseries = timeseries)

read_wave_server_v returned flag FL - requested data left (earlier) than tank contents


In [39]:
HAD_str = HAD_buf.getvalue()
HAD_dict = json.loads(HAD_str[2:-1])
HAD_dict

{'type': 'Timeseries',
 'metadata': {'intermagnet': {'imo': {'iaga_code': 'HAD',
    'name': 'Hartland',
    'coordinates': [355.5, 51.0, 0.0]},
   'reported_orientation': 'HEZF',
   'sensor_orientation': 'HDZF',
   'data_type': 'variation',
   'sampling_period': 60,
   'digital_sampling_rate': 0.01},
  'status': 200,
  'generated': '2018-09-09T19:35:28Z',
  'url': None},
 'times': ['2016-01-01T00:00:00.000Z',
  '2016-01-01T00:01:00.000Z',
  '2016-01-01T00:02:00.000Z',
  '2016-01-01T00:03:00.000Z',
  '2016-01-01T00:04:00.000Z',
  '2016-01-01T00:05:00.000Z',
  '2016-01-01T00:06:00.000Z',
  '2016-01-01T00:07:00.000Z',
  '2016-01-01T00:08:00.000Z',
  '2016-01-01T00:09:00.000Z',
  '2016-01-01T00:10:00.000Z',
  '2016-01-01T00:11:00.000Z',
  '2016-01-01T00:12:00.000Z',
  '2016-01-01T00:13:00.000Z',
  '2016-01-01T00:14:00.000Z',
  '2016-01-01T00:15:00.000Z',
  '2016-01-01T00:16:00.000Z',
  '2016-01-01T00:17:00.000Z',
  '2016-01-01T00:18:00.000Z',
  '2016-01-01T00:19:00.000Z',
  '2016-01-01T00

In [24]:
HAD_info = HAD_dict["metadata"]["intermagnet"]["imo"].copy()
HAD_info["sensor_id"] = HAD_info.pop("iaga_code")
HAD_info["longitude"], HAD_info["latitude"], _ = HAD_info.pop("coordinates")
HAD_info["website"] = "http://geomag.bgs.ac.uk/operations/hartland.html"
# Elevation obtained from their website.
HAD_info["elevation"] = 95
HAD_info["coord_sys"] = HAD_dict["metadata"]["intermagnet"]["reported_orientation"]
HAD_info["data_type"] = HAD_dict["metadata"]["intermagnet"]["data_type"]
HAD_info

{'name': 'Hartland',
 'sensor_id': 'HAD',
 'longitude': 355.5,
 'latitude': 51.0,
 'website': 'http://geomag.bgs.ac.uk/operations/hartland.html',
 'elevation': 95,
 'coord_sys': 'HEZF',
 'data_type': 'variation'}

In [25]:
add_sensor(**HAD_info)

Successfully added sensor to database!
New sensor script generated. The script can be used to insert raw sensor data to the database using write_raw_dat(). write_mag_dat() can be used to write processed magnetometer data to the database. The script can be found in this directory with the filename: HAD_dat_insert.py
Successfully generated script for inserting sensor readings for HAD!


In [40]:
import HAD_dat_insert as HAD_insert

In [42]:
HAD_magdat = pd.DataFrame({"times": HAD_dict["times"],
                    "H": HAD_dict["values"][0]["values"],
                    "E": HAD_dict["values"][1]["values"],
                    "Z": HAD_dict["values"][2]["values"]})
HAD_magdat["times"] = pd.to_datetime(HAD_magdat["times"])
HAD_magdat.set_index("times", drop=True, inplace=True)
HAD_magdat.tail()

,H,E,Z
times,,,
2016-02-29 23:56:00,19751.2,-742.9,44351.7
2016-02-29 23:57:00,19751.3,-743.2,44351.5
2016-02-29 23:58:00,19751.3,-742.9,44351.4
2016-02-29 23:59:00,19752.5,-742.9,44351.6
2016-03-01 00:00:00,19753.0,-742.3,44351.9


In [43]:
HAD_insert.write_mag_dat(HAD_magdat.to_dict('records'))